## Logistic regression
---
I'm using [this](https://www.kaggle.com/uciml/pima-indians-diabetes-database) dataset from UCI to try and predict whether a patient will develop diabetes. This is a binary classification tasks, with the target being either 0 (no diabetes) or 1 (onset of diabetes).

In [8]:
import numpy as np # linear algebra
import pandas as pd # data manipulation
import matplotlib.pyplot as plt # visualisation
import altair as alt # visualisation
import seaborn as sns # visualisation
import eli5 # for permutation importance
from pdpbox import pdp, get_dataset, info_plots # for partial dependence plotting 
from IPython.display import display # for permutation importance display
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, plot_confusion_matrix

In [9]:
diabetes_df = pd.read_csv('../data/diabetes-classification.csv')

diabetes_df.columns = [col.lower() for col in diabetes_df.columns]

First let's investigate our data. We have 7 predictive features and one target variable:
- pregnancies: number of pregnancies of the patient (they are all female)
- glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
- bloodpressure: Diastolic blood pressure (mm Hg)
- skinthickness: the skin thickness in mm at the tricep fold, a proxy indicator for BMI/obesity
- insulin: 2-Hour serum insulin (mu U/ml)
- bmi: body mass index (weight in kg)/((height in metres)^2)
- age: age in years
- outcome: 0 or 1

## Investigating the data
---
First, let's see what the columns look like:

In [11]:
diabetes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   pregnancies               768 non-null    int64  
 1   glucose                   768 non-null    int64  
 2   bloodpressure             768 non-null    int64  
 3   skinthickness             768 non-null    int64  
 4   insulin                   768 non-null    int64  
 5   bmi                       768 non-null    float64
 6   diabetespedigreefunction  768 non-null    float64
 7   age                       768 non-null    int64  
 8   outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


How well do my features correlate?

In [9]:
diabetes_df.corr()

,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,outcome
pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
bloodpressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
skinthickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
bmi,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
diabetespedigreefunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


OK, much less correlated than in the other dataset, which means we have a shot at a good prediction here.

## Data visualisation
---

In [4]:
# plt.figure(figsize=(12, 7))
# sns.boxplot(data=diabetes_df, x='outcome', y='bmi', palette='BuPu')

alt.Chart(diabetes_df).mark_bar().encode(
    x = 'outcome:O',
    y = 'count()'
).properties(width=150)

alt.Chart(...)

In [5]:
alt.Chart(diabetes_df).mark_bar(width=10).encode(
    alt.X('pregnancies:Q'),
    alt.Y('count():Q'))

alt.Chart(...)

In [6]:
alt.Chart(diabetes_df[diabetes_df['outcome']==1]).mark_bar().encode(
    x = alt.X('age:Q'),
    y = alt.Y('count():Q'),
    tooltip = [alt.Tooltip('age:Q'),
              alt.Tooltip('count()')]
). properties(
                                width=600,
                                height=300)

alt.Chart(...)

Diabetes is most prevalent in this dataset for people of age 25, 29, 31, 41.

## Splitting the data and training the models
---

In [10]:
X = diabetes_df.drop(['outcome', 'age', 'skinthickness'], axis=1)
y = diabetes_df['outcome']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

Now let's train the model, then generate some predictions and evaluate.

In [1]:
weights = []
def show_model_permutations(lin_model=None):
    """ Appends permutation importance for the labels on one model to a list.
        Prints when called with no model.
    """
    if lin_model==None:         
        for display_weights in weights:
            display(display_weights)
        return
    
    perm_importance = calc_model_perm_importance(lin_model)
    weights.append(eli5.show_weights(perm_importance, feature_names = X_test.columns.tolist()))
    
def calc_model_perm_importance(lin_model):
    """ Calculates the permutation importance for the labels on one model. """
    return eli5.sklearn.PermutationImportance(lin_model,
                                                        random_state=1).fit(X_test, y_test)

def print_results(lin_model, predictions_lin_model, solver,
                 confusion_matrix=False):
    print(solver)
    print("-----------------")
    if confusion_matrix:
        disp = plot_confusion_matrix(lin_model, X_test, y_test,
                                    display_labels=['positive', 'negative'],
                                    cmap=plt.cm.Blues,
                                    values_format= '.0f')
        disp.ax_.set_title("Confusion matrix")
        plt.show()
    print("Accuracy: {}".format(accuracy_score(y_test, predictions_lin_model)))
    print("Precision: {}".format(precision_score(y_test, predictions_lin_model)))
    print("Recall: {}".format(recall_score(y_test, predictions_lin_model)))
    print("ROC AUC: {} \n".format(roc_auc_score(y_test, predictions_lin_model)))

def show_partial_dep_plots(lin_model):
    for feat_name in X.columns:
        pdp_dist = pdp.pdp_isolate(model=lin_model, dataset=X_test, 
                                   model_features=X.columns, feature=feat_name)
        pdp.pdp_plot(pdp_dist, feat_name)
        plt.show() 
    
def parameterise_model(solver, C=1.0, max_iter=100):
    lin_model = LogisticRegression(solver=solver, random_state=0,
                              C=C, max_iter=max_iter)
    
    lin_model.fit(X_train, y_train)
    
    return lin_model

def logistic_regression():
    for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
        lin_model = parameterise_model(solver, C=0.8,
                                        max_iter=1000)
        show_model_permutations(lin_model)
        
        predictions_lin_model = lin_model.predict(X_test)
        print_results(lin_model, predictions_lin_model, solver)
    
    show_partial_dep_plots(parameterise_model(solver='newton-cg'))
    show_model_permutations() # comment out if you don't want to see permutation importance 

def main():
    logistic_regression()
    
if __name__ == '__main__':
    main()

NameError: name 'LogisticRegression' is not defined

newton-cg and lbfgs stand out as the best performers, but not by far. Decreasing C, the inverse of the regularisation strength, results in worse performance across the board.